# first exercise

We define a function which counts the number of lines for each files in specifique format in a specifique directory.
To count the number of lines for a file, we count the '\n' in the file. We read the file block by block so that it can fit in RAM.

In [1]:
import glob
def blocks(files, size=65536):
    while True:
        b = files.read(size)
        if not b:break
        yield b
        
def countLine(files):
    with open(files, 'r') as f:
        return sum(bl.count('\n') for bl in blocks(f))

def countLineForEach(directory, format_f):
    name = directory + '\\**\*.' + format_f
    for filename in glob.iglob(name, recursive=True):
        print(filename + ' :\n' + str(countLine(filename)) + '  lines')
 
directory = 'C:\\Users\\XIA\\Desktop\\amadeus_test'
format_f = 'csv'
countLineForEach(directory, format_f)

C:\Users\XIA\Desktop\amadeus_test\data\bookings.csv :
10000011  lines
C:\Users\XIA\Desktop\amadeus_test\data\searches.csv :
20390198  lines


# second exercise